In [1]:
from gensim.models.keyedvectors import KeyedVectors

model_path = 'GoogleNews-vectors-negative300.bin.gz'
w2v_model = KeyedVectors.load_word2vec_format(model_path, binary=True)

c:\users\yj\appdata\local\programs\python\python37\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [2]:
w2v_model.similarity('male', 'man')

0.36962923

In [3]:
w2v_model.n_similarity(['coding', 'interview'], ['job', 'searching'])

0.19673139

In [4]:
import sys
sys.path.append('../')
from QuestionMatcher import QuestionMatcher

qm = QuestionMatcher(w2v_model)

In [5]:
source_doc = 'I am ready to sleep now'
target_docs = 'I plan to go to bed now'
# This will return 3 target docs with similarity score
sim_scores = qm.get_similarity(source_doc, target_docs)

print(sim_scores)

0.77277756


In [16]:
source_doc = 'how to delete an invoice'
target_docs = [('1', 'delete a invoice'), ('2', 'how do i remove an invoice'), ('3' ,'purge an invoice')]
sim_scores = qm.get_recommendations(source_doc, target_docs)

print(sim_scores)

[{'name': '1', 'content': 'delete a invoice', 'score': 0.8719141}, {'name': '2', 'content': 'how do i remove an invoice', 'score': 0.83031255}, {'name': '3', 'content': 'purge an invoice', 'score': 0.7387663}]


In [17]:
import pandas as pd
# Load Movies Metadata
data = pd.read_csv('../data/leetcode_data_processed_topics3.csv', low_memory=False)
test = pd.read_csv('../data/leetcode_labels.csv', low_memory=False)

In [18]:
def is_correct(predictions, actual):
    # if there is an overlap, return True
    return not set(predictions).isdisjoint(actual)

In [26]:
target_docs = []
for index, row in data.iterrows():
    target_docs.append((row['name'], row['content']))

In [62]:
recs = {}

for index, row in data.iterrows():
    recs[row['name']] = qm.get_recommendations(row['content'], target_docs, 0.70)

In [ ]:
recs['Merge Intervals']

In [63]:
import ast

In [64]:
# Compute recommender accuracy

correct = 0
total = len(test)

for row in test.iterrows():
    question = row[1]['name']
    predictions = [obj['name'] for obj in recs[question] if obj['name'] != question]
    #print('Pred', predictions)
    actual = test[test['name'] == question].similar_questions.values[0]
    actual = ast.literal_eval(actual)
    if is_correct(actual, predictions):
        correct += 1

print('Correct = {}, Total = {}, Acc = {}'.format(correct, total, float(correct / total)))

Correct = 333, Total = 485, Acc = 0.6865979381443299


In [50]:
assert(is_correct([1,2,3], [3,4,5]) == True)
assert(is_correct(['a', 'b', 'c'], ['c', 'd', 'e']) == True)
assert(is_correct(['a', 'b', 'c'], ['x', 'd', 'e']) == False)

list_a = ['Design Circular Queue', 'Binary Search Tree Iterator', 'Insert Delete GetRandom O(1)', 'Implement Queue using Stacks', 'Peeking Iterator', 'Implement Stack using Queues', 'LRU Cache', 'Stream of Characters', 'LFU Cache', 'Remove Duplicates from Sorted Array', 'Remove Duplicates from Sorted Array II', 'Remove Element', 'My Calendar III', 'Increasing Triplet Subsequence', 'My Calendar I']
list_b = ['Design Circular Queue']
assert(is_correct(list_a, list_b) == True)

In [75]:
source = data[data['name'] == 'Longest Increasing Subsequence']['content'].values[0]